In [1]:
import pandas as pd 
import numpy as np 

In [2]:
# importing csv 
df_clean2 = pd.read_csv(r"C:\Users\tonym\Documents\Flatiron\phase_5\capstone\MBTI-ML-Social-Media-\data\df_clean2.csv")

In [3]:
df_clean2.head()

,type,Extrovert,Sensing,Thinking,Judging,posts,cleaned posts,compound_sentiment,positive_sent,negative_sent,...,ADV_avg,CONJ_avg,DET_avg,NOUN_avg,NUM_avg,PRT_avg,PRON_avg,VERB_avg,._avg,X_avg
0,INFJ,0,0,0,1,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,and moments sportscenter not top ten p...,0.9826,0.178,0.113,...,1.0,2.0,1.0,4.0,0.0,0.0,1.0,2.0,2.5,0.0
1,ENTP,1,0,1,0,'I'm finding the lack of me in these posts ver...,finding the lack these very alarming s...,0.9984,0.206,0.106,...,2.0,4.0,2.0,6.0,0.0,0.0,4.0,6.0,4.0,0.0
2,INTP,0,0,1,0,'Good one _____ https://www.youtube.com/wat...,good one course which say know...,0.9985,0.224,0.098,...,2.0,1.0,1.0,4.0,0.0,0.0,2.0,4.0,3.0,0.0
3,INTJ,0,0,1,1,"'Dear INTP, I enjoyed our conversation the o...",dear enjoyed our conversation the other ...,0.9985,0.176,0.067,...,1.5,3.0,2.0,5.0,0.0,0.0,3.0,5.0,3.0,0.0
4,ENTJ,1,0,1,1,'You're fired.|||That's another silly misconce...,you fired that another silly misconception...,0.9917,0.207,0.163,...,2.0,2.0,2.0,5.0,0.0,0.0,3.0,5.0,3.0,0.0
